# Lesson 3: Data Ingest and Wrangling

## Introduction

In this lesson we will learn to load a data file, clean it, and create visualizations with it.

In this lesson you will learn:
- Load a data file from a local source
- Clean and prepare data for analysis
- Standard data cleaning practices
- Create basic visualizations with Plotly
  

In [3]:
# Import the required libraries for data analysis and visualization
import pandas as pd          # For working with data tables (DataFrames)
import plotly.express as px   # For creating interactive charts
import re                     # For working with text patterns (regular expressions)   

## 1 Getting Data

The most basic step in any data science project is actually getting the data. The pandas library makes this very easy if you are working with tabular data (data organized in rows and columns, like a spreadsheet). 

We will be working with a CSV (Comma-Separated Values) file containing Reddit posts from the r/JMU subreddit. CSV files are one of the most common formats for storing data because they can be opened by almost any program.

We can load the Reddit CSV file using pandas' `.read_csv()` function.




### 1.1 Loading a local file

To load the CSV file locally, we use the command `pd.read_csv("filename.csv")`. The `pd` is our shorthand for pandas, and `read_csv()` is the function that reads CSV files.

In [4]:
# Load the CSV file and display it (but don't save it to a variable yet)
pd.read_csv("data/reddit_text_analysis_JMU_top_posts_default.csv")

,type,title,text,date,score
0,post,President Alger leaving to take same job at Am...,President Alger leaving to take same job at Am...,2024-03-18 12:47:10,358
1,comment,President Alger leaving to take same job at Am...,"Like him or not, he did help transform this sc...",2024-03-18 12:49:04,82
2,comment,President Alger leaving to take same job at Am...,Massive changes happening at JMU this year. Al...,2024-03-18 12:50:05,34
3,comment,President Alger leaving to take same job at Am...,Rather short tenure as JMU presidents go. He f...,2024-03-18 13:13:29,37
4,comment,President Alger leaving to take same job at Am...,He was very nice and friendly when I spoke to ...,2024-03-18 12:57:33,30
...,...,...,...,...,...
11370,comment,How do I make friends,If you wanna get touched,2025-08-22 08:05:09,3
11371,comment,How do I make friends,"Hi, I'm a junior and I just transferred to JMU...",2025-08-22 22:58:27,2
11372,comment,How do I make friends,You’ll need something that puts you in convers...,2025-08-22 11:40:34,1
11373,comment,How do I make friends,there's a fitness club that's pretty big afaik,2025-08-25 12:18:27,1


**Note** all we did here is load the file into the output screen of Jupyter Notebook, we have not actually stored it as a variable we can work with.

We can store it by having the result of the function equal a variable.

In [5]:
# Store the CSV file as a variable called 'df' (short for DataFrame)
df = pd.read_csv("data/reddit_text_analysis_JMU_top_posts_default.csv")
# Display the first 5 rows to see what our data looks like
df.head()

,type,title,text,date,score
0,post,President Alger leaving to take same job at Am...,President Alger leaving to take same job at Am...,2024-03-18 12:47:10,358
1,comment,President Alger leaving to take same job at Am...,"Like him or not, he did help transform this sc...",2024-03-18 12:49:04,82
2,comment,President Alger leaving to take same job at Am...,Massive changes happening at JMU this year. Al...,2024-03-18 12:50:05,34
3,comment,President Alger leaving to take same job at Am...,Rather short tenure as JMU presidents go. He f...,2024-03-18 13:13:29,37
4,comment,President Alger leaving to take same job at Am...,He was very nice and friendly when I spoke to ...,2024-03-18 12:57:33,30


The csv file has now been stored as a `DataFrame`. A dataframe is a lot like a spreadsheet with columns and rows, but it has some features to optimize it for data science analysis.

We can show the content of the data frame by typing `df`. The `.head()` method shows us the first 5 rows, which is useful for getting a quick look at our data.

**Note:** You do not have to use the name `df` - this is just a common convention that stands for "DataFrame".

## 2 Cleaning the DataFrame

Before you start doing anything with your data, you want to make sure you get the DataFrame cleaned up. This will make working with the data easier. 


### 2.1 DataTypes

Part of the power of Pandas is that it assumes each column contains the same type of data (all numbers, all text, all dates, etc.). This allows it to make calculations much faster. 

However, when you import from a CSV file, Pandas is not always able to automatically determine what type of data is in each column. We can check what Pandas decided by using the `.dtypes` property (note: no parentheses needed).

In [6]:
# Check what data types Pandas assigned to each column
df.dtypes

type     object
title    object
text     object
date     object
score     int64
dtype: object

Pandas was able to figure out that the **score** column contains integers, but it had trouble with the other columns. It saved these as generic `objects`, which means they could be strings, numbers, or other types of data formats. 

This can lead to problems later, so Pandas recommends converting text columns to their special string format called `StringDtype`. This makes for better storage and processing. The code below converts each column to its proper data type:

**Syntax Explanation:** To change a column's data type, we use this pattern:
- `df['column_name']` - selects the column
- `.astype('new_type')` - converts it to the new data type
- `df['column_name'] = ...` - saves the converted column back to the DataFrame

In [7]:
# Convert each column to its proper data type
df['type'] = df['type'].astype('category')        # Post type (post/comment) as category
df['title'] = df['title'].astype(pd.StringDtype())  # Title text as string
df['text'] = df['text'].astype(pd.StringDtype())    # Post content as string  
df['score'] = pd.to_numeric(df['score'])             # Score as number
df['date'] = pd.to_datetime(df['date'])              # Date as datetime
# Check our new data types
df.dtypes

type           category
title    string[python]
text     string[python]
date     datetime64[ns]
score             int64
dtype: object

Great! Now our columns have proper data types. This is basic housekeeping you should do whenever working with CSV files. While it might seem tedious, it prevents many problems later in your analysis.

## 3 Cleaning the Data

Once the dataframe is in order you will want to clean up some of the data. This is usually a recursive process. That is, you usually only figure out that there is an issue with the data when you start working on it. As you keep finding issues, you want to clean these issues earlier in your code, rather than when you run into them.

### 3.1 Removing special characters

When working with text data, a common problem is that special characters like emojis can interfere with analysis. These need to be removed because they might cause issues when using tools for **sentiment analysis** or **text processing**. You'll get better results if your data is more standardized.

In [8]:
# Define a regex (regular expression) pattern that matches common emoji characters
# This pattern covers most emoji Unicode ranges
emoji_pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]'

# Find all rows where the 'text' column contains emojis
rows_with_emojis = df[df['text'].str.contains(emoji_pattern, regex=True, na=False)]

# Show just the text column of these rows to see the emojis
rows_with_emojis[['text']]

,text
6,Does this mean SpringFest can come back!? 🚁 🔥 🛋️
47,"Lmao, I saw this meme just now right after THE..."
138,"Landing on the field would be easy, taking off..."
139,Not technically illegal as long as you don’t f...
208,Keep your eyes out for it👀
...,...
11171,My bad I've never actually been 😭 that's just ...
11173,It didn't stop me from going here 😭 I'm a big ...
11274,freshman anxiety :~) any other freshmen super ...
11279,yes I get a lot of social anxiety so I’m just ...


**Note:** In the result above, you can see there are many rows with emojis. We could manually remove these by editing each individual cell, but that would be extremely time-consuming. Instead, we'll use pandas to automatically find and remove all emojis by replacing them with empty text.

The code to do this is:
```python
df['text'] = df['text'].str.replace(emoji_pattern, '', regex=True)
```
This means: take the 'text' column, find all emojis (using our pattern), and replace them with nothing (empty string).

In [9]:
# Remove all emojis from the text column by replacing them with empty strings
df['text'] = df['text'].str.replace(emoji_pattern, '', regex=True)

In [10]:
# Check if our emoji removal worked by searching for emojis again
rows_with_emojis_cleaned = df[df['text'].str.contains(emoji_pattern, regex=True, na=False)]

# Display the results (should be empty or much fewer rows)
rows_with_emojis_cleaned[['text']]

,text


As you work with a DataFrame there are always other formatting quirks in the data you will want to take care of. It makes little sense to try to clean everything in advance and hope for the best. Likely, you'll find problems as you go and then make the fixes part of the cleaning process.

# 4 Data Visualization

Now that we have clean data with proper data types, we can create visualizations that help us understand patterns in the r/JMU subreddit. Let's start by looking at how posting activity has changed over time.

In [14]:
# Extract year-month from the date column for monthly analysis
df['year_month'] = df['date'].dt.to_period('M')
# Group the data by year-month and post type, then count entries in each group
monthly_counts = df.groupby(['year_month', 'type'], observed=True).size().reset_index(name='count')

# Convert year_month back to datetime format so Plotly can understand it
monthly_counts['year_month'] = monthly_counts['year_month'].dt.to_timestamp()

monthly_counts.head()

,year_month,type,count
0,2011-09-01,comment,19
1,2011-09-01,post,3
2,2011-10-01,comment,32
3,2011-10-01,post,5
4,2011-11-01,comment,15


In [ ]:

# Create an interactive line chart
fig = px.line(monthly_counts, 
              x='year_month',           # x-axis: time
              y='count',                # y-axis: number of posts/comments
              color='type',             # separate lines for posts vs comments
              title='Number of Posts and Comments per Month on r/JMU',
              labels={'count': 'Number of Posts/Comments', 'year_month': 'Year-Month'},
              markers=True)             # add dots to the lines

# Customize the chart appearance
fig.update_layout(
    xaxis_title="Year-Month",
    yaxis_title="Number of Posts/Comments",
    legend_title="Type"
)

# Display the interactive chart
fig.show()

### 4.1 Keyword Analysis: Do Certain Topics Get Higher Scores?

Now let's analyze whether posts containing certain keywords tend to get higher scores (more upvotes). This can tell us what topics are most popular or engaging in the JMU community.

In [12]:
# Define keywords to analyze - common JMU-related topics
keywords = ['tuition', 'covid', 'party', 'football', 'class', 'library', 'campus']

# Create a helper function to check if text contains a keyword (case-insensitive)
def contains_keyword(text, keyword):
    # Handle missing/empty text
    if pd.isna(text):
        return False
    # Convert both to lowercase to make the search case-insensitive
    return keyword.lower() in text.lower()

# Store results for each keyword
keyword_scores = []

for keyword in keywords:
    # Create a temporary True/False mask showing which posts contain this keyword
    # We don't add this to the DataFrame to keep it clean
    has_keyword_mask = df['text'].apply(lambda x: contains_keyword(x, keyword))
    
    # Calculate the average score for posts containing this keyword
    # Use the mask to filter posts that contain the keyword
    avg_score = df[has_keyword_mask]['score'].mean()
    
    # Store the results
    keyword_scores.append({
        'keyword': keyword,
        'score': avg_score
    })

# Convert results to a DataFrame for easier plotting
keyword_df = pd.DataFrame(keyword_scores)

# Create a bar chart showing average scores by keyword
fig = px.bar(keyword_df, 
             x='keyword', 
             y='score',
             title='Average Post Scores by Keyword',
             labels={'score': 'Average Score', 'keyword': 'Keyword'})

# Customize chart appearance
fig.update_layout(
    xaxis_title="Keywords",
    yaxis_title="Average Score",
    showlegend=False  # No legend needed for single series
)

# Display the chart
fig.show()



## 5 Saving Your Work

Great work! We now have a clean DataFrame that we can use for further analysis. It's a good practice to save your cleaned data so you don't have to repeat all the cleaning steps every time you want to work with it.

There are two main ways to save a DataFrame:

Whenever you run code in Jupyter, the notebook automatically stores the values of the variables. When you restart, these values are deleted. It is a good practice to save the dataframe variable once you have performed a lot of operations on it. This prevents us from having to run the above code everytime. 

There are a couple of ways we can save this. We can use the pandas function `to_csv()`, which converts this to a csv file. The advantage of a CSV file is that any computer can read them.
</br> 
There are several issues with CSV files:
- They tend to be big
- Read and write times can be slow
- Will add an empty index column if you are not careful
- Unless you specifically indicate the column types, the dtype will get lost. This is a huge pain.

Alternatively, we can also save this to a `.pickle` file.

**Advantages**

- Smaller
- Faster
- Keeps dtypes

**Disadvantages**

- Requires Python to open
  
  

Saving to pickle file is incredibly simple. 

In [11]:
df.head()

,type,title,text,date,score,year_month
0,post,President Alger leaving to take same job at Am...,President Alger leaving to take same job at Am...,2024-03-18 12:47:10,358,2024-03
1,comment,President Alger leaving to take same job at Am...,"Like him or not, he did help transform this sc...",2024-03-18 12:49:04,82,2024-03
2,comment,President Alger leaving to take same job at Am...,Massive changes happening at JMU this year. Al...,2024-03-18 12:50:05,34,2024-03
3,comment,President Alger leaving to take same job at Am...,Rather short tenure as JMU presidents go. He f...,2024-03-18 13:13:29,37,2024-03
4,comment,President Alger leaving to take same job at Am...,He was very nice and friendly when I spoke to ...,2024-03-18 12:57:33,30,2024-03


In [12]:
df.to_pickle('data/jmu_reddit.pickle')

The code below should place the file in your working directory.